# Trabalho Prático 2

##### **Nome:** André Luiz Moreira Dutra
##### **Matrícula:** 2019006345

Neste trabalho, foi implementado um modelo baseado em BERT para a tarefa de POS-tagging. A tarefa de POS-tagging consiste em, dada uma frase, classificar cada palavra segundo sua classe gramatical na língua. O modelo implementado neste realiza a tarefa de POS-tagging para a língua portuguesa.

Primeiramente, vamos instalar e importar todos os pacotes necessários para este trabalho:

In [1]:
#Instalação dos pacotes necessários
! pip install transformers[torch] datasets evaluate seqeval

In [2]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification

import evaluate
from numpy import argmax

import torch
from transformers import AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments

from torch.utils.data import DataLoader

2023-12-13 15:03:14.599823: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Dataset Utilizado

O dataset utilizado é o Mac-Morpho, um dataset projetado especificamente para a tarefa de POS-tagging para a língua portuguesa. O dataset consiste em cerca de 50.000 entradas, com cada uma contendo uma sentença tokenizada e a respectiva lista de POS-tags de cada token da sentença. O dataset é dividido em 76%-4%-20% para treino, validação e teste. Cada token é classificado em uma de 26 POS-tags diferentes.

Neste trabalho, o Mac-Morpho foi carregado diretamente do huggingface conforme o seguinte código:

In [3]:
macmorpho_dataset = load_dataset('mac_morpho')

label_list = macmorpho_dataset['train'].features['pos_tags'].feature.names
id2label = {idx: label for idx, label in enumerate(label_list)}
label2id = {label: idx for idx, label in enumerate(label_list)}

macmorpho_dataset['train'][0]

Generating train split:   0%|          | 0/37948 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9987 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1997 [00:00<?, ? examples/s]

Dataset mac_morpho downloaded and prepared to /home/andre/.cache/huggingface/datasets/mac_morpho/default/3.0.0/c87ce539abd72e76dd6b65403173c3e206ef0610d61bd7f487c438f2541a246b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Sequence(feature=ClassLabel(names=['PREP+PROADJ', 'IN', 'PREP+PRO-KS', 'NPROP', 'PREP+PROSUB', 'KC', 'PROPESS', 'NUM', 'PROADJ', 'PREP+ART', 'KS', 'PRO-KS', 'ADJ', 'ADV-KS', 'N', 'PREP', 'PROSUB', 'PREP+PROPESS', 'PDEN', 'V', 'PREP+ADV', 'PCP', 'CUR', 'ADV', 'PU', 'ART'], id=None), length=-1, id=None)

## Modelo utilizado e tokenização
O modelo a ser treinado se trata de um fine-tuning do BERTimbau. O BERTimbau nada mais é do que uma arquitetura BERT pré-treinada com dados em português brasileiro. Como o nosso dataset se constitui de palavras em português brasileiro, o BERTimbau se mostrou como a alternativa mais interessante de modelo pré-treinado. Tendo o BERTimbau como base, foi treinado um modelo de classificação de tokens, o qual é adaptado para a classificação de palavras.

O BERTimbau possui um tokenizer próprio, o qual também usamos neste trabalho. No entanto, surge o problema de que o tokenizer muitas vezes converte uma palavra em múltiplos tokens, tornando as POS-tags inconsistentes com o tamanho do vetor de tokens.

Desse modo, como estamos utilizando um modelo de classificação de tokens e a tokenização não pode ser modificada devido ao pré-treinamento do modelo, utilizamos o tokenizador original e, para cada token gerado a partir de uma palavra do dataset, atribuímos a POS-tag da palavra aos tokens. Ao fim do treinamento, podemos transformar de volta as previsões dos tokens em previsões de palavras selecionando, para cada palavra, os logits de predição dos seus tokens correspondentes, tirando a média e utilizando o resultado para prever a POS-tag da palavra.

O código a seguir tokeniza a base de dados e adapta o vetor de labels original à sua versão tokenizada. Os paddings são feitos sob demanda a cada batch utilizando um DataCollator, já que fazer o padding batch-a-batch diminuímos o tamanho médio dos vetores, melhorando a eficiência de memória.

In [4]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

def align_labels(words, labels, tokenizer):
  token_labels = [[label for token in tokenizer.tokenize(word)] for word, label in zip(words, labels)]
  return [-100] + sum(token_labels, []) + [-100]

def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], is_split_into_words=True)
  tokenized_inputs['labels'] = [align_labels(words, labels, tokenizer) for words, labels in zip(examples['tokens'], examples['pos_tags'])]

  return tokenized_inputs

tokenized_dataset = macmorpho_dataset.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Loading cached processed dataset at /home/andre/.cache/huggingface/datasets/mac_morpho/default/3.0.0/c87ce539abd72e76dd6b65403173c3e206ef0610d61bd7f487c438f2541a246b/cache-a14d891a915977be.arrow
Loading cached processed dataset at /home/andre/.cache/huggingface/datasets/mac_morpho/default/3.0.0/c87ce539abd72e76dd6b65403173c3e206ef0610d61bd7f487c438f2541a246b/cache-d1d917f37c9bfb0c.arrow
Loading cached processed dataset at /home/andre/.cache/huggingface/datasets/mac_morpho/default/3.0.0/c87ce539abd72e76dd6b65403173c3e206ef0610d61bd7f487c438f2541a246b/cache-ebe39e72967b879a.arrow


## Métricas de Avaliação Utilizadas na Validação

Como forma de validar o modelo gerado, a seguinte função foi implementada. Ela basicamente utiliza o módulo seqeval para comparar as sequências de POS-tags geradas com as sequências verdadeiras, gerando acurácia, precisão, revocação e F1. Como o seqeval é projetado para a tarefa de NER, adicionamos o prefixo "B-" para que o método as reconheça. A adaptação é efetiva para POS-tags e documentada como alternativa de uso para tags não-NER no próprio github do pacote (https://github.com/chakki-works/seqeval/issues/75). A função é passada ao treinador do modelo e chamada a cada época.

Note que as métricas calculadas são em relação à tarefa de POS-tagging de tokens, e não de palavras. Embora o mapeamento do problema de palavras para tokens tenha sido feito de maneira que ambos sejam muito semelhantes, ainda se trata de problemas diferentes. No entanto, no contexto de validação do modelo durante o treinamento, basta calcular as métricas para a classificação de tokens, já que o modelo treinado em si é um TokenClassifier.

In [5]:
seqeval = evaluate.load('seqeval')

def compute_metrics(p):
  prediction_logits, true_labels_idxs = p
  predicted_labels_idxs = argmax(prediction_logits, axis=2)

  predicted_labels = [[f'B-{label_list[pred_idx]}' for pred_idx, true_idx in zip(predicted_idxs, true_idxs) if true_idx != -100] for predicted_idxs, true_idxs in zip(predicted_labels_idxs, true_labels_idxs)]
  true_labels = [[f'B-{label_list[true_idx]}' for true_idx in true_idxs if true_idx != -100] for true_idxs in true_labels_idxs]

  results = seqeval.compute(predictions=predicted_labels, references=true_labels)

  return {
    "precision": results["overall_precision"],
    "recall": results["overall_recall"],
    "f1": results["overall_f1"],
    "accuracy": results["overall_accuracy"],
  }

## Hiperparâmetros do Modelos

Conforme mencionado anteriormente, o modelo base utilizado é o BERTimbau. Como forma de otimizar o espaço alocado na GPU, a versão com pesos do tipo float32 foi utilizada. Como parâmetros do modelo, estou usando taxa de aprendizado 0.00002, 3 épocas, weight decay de 0.1 e tamanho de batch igual a 4. O tamanho de batch foi escolhido devido às limitações de memória, e a taxa de aprendizado, número de épocas e weight decay foram escolhidos usando como referência os minicursos de huggingface de Token Classification utilizando modelos baseados no BERT.

No código a seguir, temos a instanciação do modelo, definição dos hiperparâmetros e inicialização do treinador com os dados de treino e validação. Na célula seguinte, o modelo é treinado.

In [6]:
torch.cuda.empty_cache()

model = AutoModelForTokenClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', torch_dtype=torch.float32, num_labels = len(label_list), id2label = id2label, label2id = label2id)

training_args = TrainingArguments(
    output_dir="model_results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model check

In [7]:
trainer.train()
results = trainer.evaluate()
results

/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/28461 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5574, 'learning_rate': 1.9648642001335163e-05, 'epoch': 0.05}
{'loss': 0.1406, 'learning_rate': 1.929728400267032e-05, 'epoch': 0.11}
{'loss': 0.1133, 'learning_rate': 1.8945926004005485e-05, 'epoch': 0.16}
{'loss': 0.1158, 'learning_rate': 1.8594568005340643e-05, 'epoch': 0.21}
{'loss': 0.099, 'learning_rate': 1.8243210006675805e-05, 'epoch': 0.26}
{'loss': 0.0943, 'learning_rate': 1.7891852008010962e-05, 'epoch': 0.32}
{'loss': 0.0863, 'learning_rate': 1.7540494009346124e-05, 'epoch': 0.37}
{'loss': 0.0929, 'learning_rate': 1.7189136010681285e-05, 'epoch': 0.42}
{'loss': 0.0826, 'learning_rate': 1.6837778012016446e-05, 'epoch': 0.47}
{'loss': 0.0791, 'learning_rate': 1.6486420013351604e-05, 'epoch': 0.53}
{'loss': 0.0817, 'learning_rate': 1.6135062014686765e-05, 'epoch': 0.58}
{'loss': 0.0815, 'learning_rate': 1.5783704016021927e-05, 'epoch': 0.63}
{'loss': 0.0774, 'learning_rate': 1.5432346017357088e-05, 'epoch': 0.69}
{'loss': 0.0829, 'learning_rate': 1.5080988018692246e

  0%|          | 0/500 [00:00<?, ?it/s]

...

/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PREP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: N seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: V seems not to be NE tag.

{'eval_loss': 0.07421258091926575, 'eval_precision': 0.9804769087955442, 'eval_recall': 0.9810176182045105, 'eval_f1': 0.9807471889735219, 'eval_accuracy': 0.9834984901969124, 'eval_runtime': 12.009, 'eval_samples_per_second': 166.292, 'eval_steps_per_second': 41.636, 'epoch': 3.0}
{'train_runtime': 3552.2484, 'train_samples_per_second': 32.048, 'train_steps_per_second': 8.012, 'train_loss': 0.06540923733297585, 'epoch': 3.0}


  0%|          | 0/500 [00:00<?, ?it/s]

/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADV seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PREP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: N seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PU seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/andre/miniconda3/envs/notebook_env/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: V seems not to be NE tag.

{'eval_loss': 0.06989305466413498,
 'eval_precision': 0.9771771597598817,
 'eval_recall': 0.9780280381969639,
 'eval_f1': 0.9776024138331205,
 'eval_accuracy': 0.9813507421426445,
 'eval_runtime': 11.9688,
 'eval_samples_per_second': 166.851,
 'eval_steps_per_second': 41.775,
 'epoch': 3.0}

## Modelo Resultante

Conforme observado no treinamento acima, os resultados sugerem muito bons resultados, com o modelo apresentando acurácia de mais de 98% de validação. No entanto, precisamos verificar o desempenho de fato aplicando o teste. Para isso, temos um modelo treinado com os hiperparâmetros mencionados anteriormente salvos no diretório definido na variável best_model.

O código a seguir faz o carregamento deste modelo, junto ao tokenizer e o dataset. Carregamos o dataset e o tokenizer novamente para que ambos estejam no mesmo ambiente do modelo e para que o dataset original seja utilizado para o teste, e não o tokenizado utilizado para o treinamento.

In [2]:
best_model = 'model_results/checkpoint-28461'
loaded_model = AutoModelForTokenClassification.from_pretrained(best_model)
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False,  return_tensors="pt")

macmorpho_dataset = load_dataset('mac_morpho')
label_list = macmorpho_dataset['train'].features['pos_tags'].feature.names

macmorpho_dataset['test'][0]

Found cached dataset mac_morpho (/home/andre/.cache/huggingface/datasets/mac_morpho/default/3.0.0/c87ce539abd72e76dd6b65403173c3e206ef0610d61bd7f487c438f2541a246b)


  0%|          | 0/3 [00:00<?, ?it/s]

{'id': '0', 'tokens': ['Salto', 'sete'], 'pos_tags': [14, 12]}

## Inferência e Teste

Conforme explicado nas células anteriores, o modelo é o classificador de tokens, e não de palavras. Para isso, na inferência aplicamos a tokenização e obtemos os logits de predição de cada POS-tag para cada token. Em seguida, para cada palavra da entrada, verificamos quantos tokens são gerados para aquela palavra após a tokenização. Então, selecionamos os logits correspondentes aos k tokens oriundos daquela palavra e tiramos a média, gerando assim o logit de predição da palavra. Fazemos isso para cada palavra da entrada. Com os logits de cada palavra, selecionamos a POS-tag de maior probabilidade e atribuímos à palavra.

O teste do modelo foi feito utilizando a base original, e portanto as labels não foram adaptadas para o vetor de tokens. Como forma de avaliar o modelo de maneira fiel à sua função de inferência, o mesmo foi testado para a classificação de palavras utilizando a técnica definida anteriormente, ao contrário da avaliação da classificação token-a-token. Desse modo, para cada sentença da entrada, realizamos a inferência usando a função acima e, ao fim, calculamos as métricas das labels previstas em relação às labels originais, sem adaptação, da base.

O código a seguir implementa as funções de inferência e teste, e testa o modelo implementado contra a base de teste do Mac-Morph:

In [35]:
seqeval = evaluate.load('seqeval')

#Função de inferência de POS-tags dada uma sentença, usando o modelo treinado
def predict_POS_tags(sentence, model, tokenizer, is_split_into_words = False):
  words = sentence if is_split_into_words else sentence.split()
  input = tokenizer(sentence, return_tensors='pt', is_split_into_words=is_split_into_words)
  
  #Realizamos a inferência dos logits (probabilidade de cada label)
  with torch.no_grad():
    logits = model(**input).logits[0]

  logits = logits[1:-1]

  #Convertemos tokens para palavras. Para cada palavra, selecionamos os k tokens aos quais ela corresponde após a tokenização e tiramos a média dos logits
  word_logits = []
  token_nums = [len(tokenizer.tokenize(word)) for word in words if len(tokenizer.tokenize(word)) != 0]
  tok_idx = 0

  for token_num in token_nums:
    word_logit = torch.mean(logits[tok_idx: tok_idx + token_num], dim=0)
    word_logits.append(word_logit)
    tok_idx += token_num

  #Para cada palavra, selecionamos o índice da label de maior probabilidade, convertemos o índice para o nome da label e geramos a saída
  predicted_labels_idxs = argmax(word_logits, axis=1)

  predicted_labels = [label_list[idx] for idx in predicted_labels_idxs]

  return predicted_labels

#Função de teste do modelo. Este teste leva em conta a inferência de POS tags por palavra, não por token como durante o treino
def test_POS_tags_model(test_dataset, model, tokenizer, is_split_into_words = True):
  true_POS_tags = [[f'B-{label_list[label_idx]}' for label_idx in label] for label in test_dataset['pos_tags']]
  predicted_POS_tags = [predict_POS_tags(words, model, tokenizer, is_split_into_words) for words in test_dataset['tokens']]
  predicted_POS_tags = [[f'B-{lab}' for lab in label] for label in predicted_POS_tags]

  results = seqeval.compute(predictions=predicted_POS_tags, references= true_POS_tags)

  overall_results = {
    "precision": results["overall_precision"],
    "recall": results["overall_recall"],
    "f1": results["overall_f1"],
    "accuracy": results["overall_accuracy"],
  }

  per_tag_precisions = {tag: results[tag]['precision'] for tag in label_list}

  return overall_results, per_tag_precisions

overall_results, per_tag_precisions = test_POS_tags_model(macmorpho_dataset['test'], loaded_model, tokenizer)
overall_results

{'precision': 0.9813144366019521,
 'recall': 0.9813144366019521,
 'f1': 0.9813144366019521,
 'accuracy': 0.9813144366019521}

## Resultados

Como podemos ver, o modelo se mostrou extremamente efetivo na tarefa de POS-tagging, apresentando acurácia acima de 98% no teste. A precisão, recall e f1 acima de 97% sugerem, também, que este resultado não é desbalanceado entre as POS-tags classificadas, por serem valores também extremamente altos. Vamos observar a precisão de cada label no código a seguir:

In [48]:
import pandas as pd

s = pd.Series(per_tag_precisions)
df = pd.DataFrame(s, columns=['precision'])
sorted_df = df.sort_values('precision', ascending = False)
print('Precisões totais:')
display(sorted_df)

print('Maiores precisões:')
display(sorted_df.iloc[:5])

print('Menores precisões:')
display(sorted_df.iloc[-5:])

Precisões totais:


,precision
PREP+PROPESS,1.000000
PU,0.999777
PREP+PROADJ,0.996753
CUR,0.996622
V,0.995837
PROPESS,0.994753
PREP+ART,0.993051
ART,0.989786
KC,0.988649
N,0.981395


Maiores precisões:


,precision
PREP+PROPESS,1.000000
PU,0.999777
PREP+PROADJ,0.996753
CUR,0.996622
V,0.995837


Menores precisões:


,precision
PREP+PROSUB,0.906667
PDEN,0.885740
PREP+PRO-KS,0.870968
ADV-KS,0.847826
IN,0.480263


['PREP+PROADJ', 'IN', 'PREP+PRO-KS', 'NPROP', 'PREP+PROSUB', 'KC', 'PROPESS', 'NUM', 'PROADJ', 'PREP+ART', 'KS', 'PRO-KS', 'ADJ', 'ADV-KS', 'N', 'PREP', 'PROSUB', 'PREP+PROPESS', 'PDEN', 'V', 'PREP+ADV', 'PCP', 'CUR', 'ADV', 'PU', 'ART']


## Precisões Por Classe

Observando as precisões de cada classe, podemos observar que, das 26 POS-tags, 21 apresentaram precisão acima de 90%, indicando um ótimo resultado geral para a maioria das tags. Observando as 5 maiores precisões, todas foram maiores que 99%, para as tags correspondentes às respectivas classes gramaticais:

* Preposição + Pronome Pessoal (PREP+PROPESS), ex. nela, dele
* Pontuação (PU) ex. !, ?, .
* Preposição + Pronome Adjetivo (PREP+PROADJ), ex. nesse, daquele
* Símbolo de Moeda Corrente (CUR), ex. R$, £
* Verbo (V), ex. Cantarei, Falhou

É notável que as quatro tags de maior precisão correspondem a palavras/tokens que, num geral, independem do contexto para serem classificadas. Embora uma mesma palavra possa assumir diferentes classes gramaticais em diferentes contextos, algumas palavras são usadas exclusivamente, ou majoritariamente, para uma única classe específica. Um bom exemplo disso que pode ser observado nas classes acima são os símbolos de moeda corrente e as pontuações. Os contextos nos quais os tokens correspondentes a estas classes são utilizados são notavelmente quase que exclusivos a elas. O mesmo ocorre para as preposições. Os verbos, embora possam assumir outros papéis gramaticais (ex. em "O cantar dos pássaros" a palavra "cantar" assume posição de substantivo), também assumem posição majoritariamente de verbos.

Vamos observar agora as preposições de menor precisão:

* Preposição + Pronome Substantivo (PREP+PROSUB) ex. "Disso_PREP+POSUB você entende"
* Palavra Denotativa (PDEN), ex. "Então_PDEN quem foi?"
* Preposição + Pronome Conectivo Subordinativo (PREP+PRO-KS) ex. "A cidade à=qual_PREP+PRO-KS fui"
* Advérbio Conectivo Subordinativo(ADV-KS) ex. "Sei como_ADV-KS resolver o problema"
* Interjeição (IN), ex. "Bom dia", "nossa", "Ô de casa"

Como podemos ver acima, as classes de menor precisão são, também, as que mais dependem do contexto. Por exemplo, "Palavra Denotativa" é a descrição dada a uma palavra que geralmente assumiria posição de advérbio, mas que no contexto da frase não modifica nenhum adjetivo ou verbo. Desse modo, todas os tokens correspondentes a palavras-denotativas podem também assumir posição de advérbio. O mesmo ocorre para advérbios conectivos subordinativos, que são advérbios que assumem também o papel gramatical de introduzir uma oração subordinativa. Algo semelhante ocorre com a Preposição + Pronome Substantivo e a Preposição + Pronome Conectivo Subordinativo, que correspondem aos mesmos tokens de Preposição + Pronome (ex. PREP+PROADJ, PREP+PRO) porém no contexto gramatical específico de assumir posição substantiva (como sujeito ou objeto) ou introduzir orações subordinativas. Com isso, é esperado que a precisão destas classes, comparada as demais, seja mais baixa, uma vez que elas dependem quase que exclusivamente dos mecanismos de atenção para a classificação, já que compartilham possíveis tokens comm muitas outras classes. Mesmo assim, para as classes citadas, o modelo ainda teve precisão acima de 84%, o que ainda é considerado bom, evidenciando a importância dos mecanismos de atenção neste caso.

É notável, no entanto, que a classe de interjeições teve uma precisão consideravelmente baixa, não só comparada às demais classes, como de maneira geral tamém, ficando abaixo de 50%, que seria a predição aleatória. Por um lado, as interjeições sofrem dos mesmos problemas mencionados nas classes anteriores, uma vez que, com a excceção de gírias, todas as interjeições e locuções interjeitivas são formadas por palavras que podem assumir outras classes gramaticais. Isso ocorre pois interjeições correspondem a palavras ou frases usadas para manifestar emoções súbitas. Então qualquer palavra ou frase pode assumir o papel de uma interjeição, inclusive palavras quase que exclusivas a uma classe gramatical específica, como mencionado ao descrever as maiores precisões. No entanto, também é possível que o método de tokenização tenha prejudicado a representação do modelo para interjeições. Por um lado, por algumas interjeições corresponderem a gírias, elas não estão no vocabulário do BERTimbau, sendo decompostas em sequências menores e mesmo caractere-a-caractere. Por outro, é muito comum que interjeições ocorram no começo da frase, e o BERTimbau só possui palavras completas em letras minúsculas em seu vocabulário. Desse modo, sempre que uma palavra começa com uma letra maiúscula, ela é decomposta em sub-tokens, o que pode ter sido um fator limitante no caso das interjeições.

Por fim, podemos concluir que o modelo proposto apresentou um ótimo desempenho não só de maneira geral, como também classe-a-classe, apresentando precisão acima de 90% para 80% das tags e precisão acima de 80% para 96% das tags. Podemos concluir, portanto, que o modelo resolve de maneira efetiva a tarefa de POS-tagging de sentenças da língua portuguesa na grande maioria dos casos de uso.